In [1]:
import pandas as pd

In [2]:
!pip install elasticsearch

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from __future__ import unicode_literals
from elasticsearch import Elasticsearch

import json

def get_elastic():
    return Elasticsearch(
        [{'host': 'localhost', 'port': 9200, 'scheme': 'http'}], 
        timeout=30, 
        max_retries=1, 
        retry_on_timeout=False
        )

# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

def query_search(q_dic, index_name, n_search=1000, elastic=None):
    '''
    q_dic: query dictionary in Elastic format ready to search 
    index_name: index in Elastic where to search
    '''
    if elastic is None:
        elastic = get_elastic()
    #body = json.dumps(q_dic, ensure_ascii=False)
    search_result = elastic.search(index=index_name, body=q_dic, size=n_search, from_=0)

    return search_result

def result2list_unique(result_dict, field):
    lst = []
    if result_dict['hits']['total']['value'] > 0:
        for hit in result_dict['hits']['hits']:
            if hit['_source'][field] not in lst: 
                lst.append(hit['_source'][field])
    return lst

def result2list_(result_dict, field):
    lst = []
    if result_dict['hits']['total']['value'] > 0:
        for hit in result_dict['hits']['hits']:
            lst.append(hit['_source'][field])
    return lst


def get_umls2sab_query(source_id, target_sab, lang):
    '''
    source_id: UMLS ID (CUI) to search
    target_sab: ontology mapped to this ID
    '''
    q_dic = {
        "query": { 
            "bool": { 
            "must": [
                {"match": {"CODE": source_id}},
                {"match": {"SAB": target_sab}}, 
                {"match": {"LAT": lang}}
                ]
                    }
                }
            }
    return q_dic

def query_and_search(code, index): 
    result_list = []
    q_dic = get_umls2sab_query(code, 'SCTSPA', 'SPA')
    result = query_search(q_dic, index)
    if result['hits']['total']['value'] > 0:
        for hit in result['hits']['hits']: 
            result_list.append(hit['_source'])

    return result_list

def get_umlstext_query(term, index, lang):
    '''
    source_id: UMLS ID (CUI) to search
    target_sab: ontology mapped to this ID
    '''
    q_dic = {
        "query": { 
            "bool": { 
            "must": [
                {"match": {
                  "STR": term
                }}, 
                {"match": {"LAT": lang}}
                ]
                    }
                }
            }
    return q_dic

def query_and_search_text(term, index): 
    result_list = []
    q_dic = get_umlstext_query(term, 'UMLS', 'ENG')
    result = query_search(q_dic, index)
    if result['hits']['total']['value'] > 0:
        for hit in result['hits']['hits']: 
            result_list.append(hit['_source'])

    return result_list

In [ ]:
#bg_test_file_sections.tsv

In [10]:
df_sections = pd.read_csv('../bg_test_file_sections.tsv', sep='\t')
df_sections.head()

,text,entities,filename,sentence_start,sentence_end
0,"дългогодишна хипертония , има често главоболие...",NaN,bg_text_0.txt,0,239
1,"съобщава за сухота в устата предимно сутрин , ...",NaN,bg_text_1.txt,0,317
2,"има често главоболие , световъртеж , сърцебиен...",NaN,bg_text_2.txt,0,270
3,"болки в корема , често уриниране , парене , не...",NaN,bg_text_3.txt,0,120
4,пушач . има хроничен задух от години и ползва ...,NaN,bg_text_4.txt,0,470


In [11]:
df_sections_idx = df_sections.set_index('filename')

In [12]:
df_sections_idx.loc['bg_text_0.txt']

text              дългогодишна хипертония , има често главоболие...
entities                                                        NaN
sentence_start                                                    0
sentence_end                                                    239
Name: bg_text_0.txt, dtype: object

In [38]:
import pandas as pd

df_terms = pd.read_csv("../bg_all_matches.tsv", sep='\t', header=0)

In [50]:
df_terms.head(50)

,filename,term,start,end,type,normalized_type,normalized_term,file_start,file_end,original_filename,links,mark,label,semantic_rel,off0,off1,span,code
0,bg_text_0.txt,дългогодишна хипертония,0,23,Disease,disease,long-term hypertension,0,23,bg_text_0.txt,UMLS:C0443252,T1,ENFERMEDAD,EXACT,0,23,дългогодишна хипертония,UMLS:C0443252
1,bg_text_0.txt,има често главоболие,26,46,Symptom,symptom,has frequent headaches,26,46,bg_text_0.txt,UMLS:C0948396,T1,ENFERMEDAD,EXACT,26,46,има често главоболие,UMLS:C0948396
2,bg_text_0.txt,сърцебиене,49,59,Symptom,symptom,palpitations,49,59,bg_text_0.txt,UMLS:C0030252,T1,ENFERMEDAD,EXACT,49,59,сърцебиене,UMLS:C0030252
3,bg_text_0.txt,сърцебиене,137,147,Symptom,symptom,palpitations,137,147,bg_text_0.txt,UMLS:C0030252,T1,ENFERMEDAD,EXACT,137,147,сърцебиене,UMLS:C0030252
4,bg_text_0.txt,лесно заморяване,62,78,Symptom,symptom,easily fatigued,62,78,bg_text_0.txt,UMLS:C1859691,T1,ENFERMEDAD,EXACT,62,78,лесно заморяване,UMLS:C1859691
5,bg_text_0.txt,продължават оплакванията от стягане в сърдечна...,79,134,Symptom,symptom,continue complaints of chest tightness,79,134,bg_text_0.txt,UMLS:C0232292,T1,ENFERMEDAD,EXACT,79,134,продължават оплакванията от стягане в сърдечна...,UMLS:C0232292
6,bg_text_0.txt,гадене,150,156,Symptom,symptom,nausea,150,156,bg_text_0.txt,UMLS:C0027497,T1,ENFERMEDAD,EXACT,150,156,гадене,UMLS:C0027497
7,bg_text_0.txt,доказан карцином на простатата,159,189,Disease,disease,proven prostate cancer,159,189,bg_text_0.txt,UMLS:C0456369,T1,ENFERMEDAD,EXACT,159,189,доказан карцином на простатата,UMLS:C0456369
8,bg_text_0.txt,насочва се за хоспитализация за химиотерапия,192,236,Procedure,procedure,is being referred for hospitalization for chem...,192,236,bg_text_0.txt,UMLS:C1830395,T1,ENFERMEDAD,EXACT,192,236,насочва се за хоспитализация за химиотерапия,UMLS:C1830395
9,bg_text_1.txt,сухота в устата,12,27,Symptom,symptom,dryness in the mouth,12,27,bg_text_1.txt,UMLS:C0043352,T1,ENFERMEDAD,EXACT,12,27,сухота в устата,UMLS:C0043352


In [21]:
def get_row_text(row):
    #print(row)
    filename = row['note_id'] + '.txt'
    section = df_sections_idx.loc[filename]
    return section['text'][row['start']:row['end']]

In [22]:
#df_terms['text'] = df_terms.apply(lambda x: get_row_text(x), axis=1)


In [60]:
links = []
for index, row in df_terms.iterrows():
    text = row['normalized_term'].replace('_', ' ')
    results = query_and_search_text(text, 'umls')
    link = ''
    if len(results) > 0:        
        link = 'UMLS:' + results[0]['CUI']
    
    links.append(link)

/tmp/ipykernel_227013/124867022.py:7: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  return Elasticsearch(


In [61]:
df_terms['links'] = links
df_terms.head()

,filename,term,start,end,type,normalized_type,normalized_term,file_start,file_end,original_filename,links,mark,label,semantic_rel,off0,off1,span,code
0,bg_text_0.txt,дългогодишна хипертония,0,23,Disease,disease,long-term hypertension,0,23,bg_text_0.txt,UMLS:C0443252,T1,ENFERMEDAD,EXACT,0,23,дългогодишна хипертония,UMLS:C0443252
1,bg_text_0.txt,има често главоболие,26,46,Symptom,symptom,has frequent headaches,26,46,bg_text_0.txt,UMLS:C0948396,T1,ENFERMEDAD,EXACT,26,46,има често главоболие,UMLS:C0948396
2,bg_text_0.txt,сърцебиене,49,59,Symptom,symptom,palpitations,49,59,bg_text_0.txt,UMLS:C0030252,T1,ENFERMEDAD,EXACT,49,59,сърцебиене,UMLS:C0030252
3,bg_text_0.txt,сърцебиене,137,147,Symptom,symptom,palpitations,137,147,bg_text_0.txt,UMLS:C0030252,T1,ENFERMEDAD,EXACT,137,147,сърцебиене,UMLS:C0030252
4,bg_text_0.txt,лесно заморяване,62,78,Symptom,symptom,easily fatigued,62,78,bg_text_0.txt,UMLS:C1859691,T1,ENFERMEDAD,EXACT,62,78,лесно заморяване,UMLS:C1859691


In [62]:
df_test = df_terms

In [63]:
#df_test['filename'] = ''
df_test['mark'] = 'T1'
df_test['label'] = 'ENFERMEDAD'
df_test['semantic_rel'] = 'EXACT'
df_test['off0'] = df_test['start']
df_test['off1'] = df_test['end']
df_test['span'] = df_test['term']
df_test['code'] = df_test['links']

#for index, row in df_test.iterrows():
#    df_test.at[index, 'filename'] = str(row['document_id']) + '_' + str(row['sentence_id'])

In [64]:
df_test_new = df_test[['filename','mark','label','off0','off1',
                                           'span', 'code', 'semantic_rel']]

In [65]:
df_test_new[df_test_new['span'] == 'no-match'].index
#df_test_new = df_test_new.drop(df_test_new[df_test_new['span'] == 'no-match'].index)

Int64Index([], dtype='int64')

In [52]:
#df_test_new['add'] = False
#df_test_new['sort_column'] = df_test_new['span'].str.len()
#df_test_new = df_test_new.sort_values(by=['filename', 'sort_column'], ascending=False)

In [53]:
#df_test_groups = df_test_new.groupby(by=["filename"])

#for group in df_test_groups:
#    added_indices = []
#    for index, row in group[1].iterrows():
#        if row['off0'] not in added_indices and row['off1'] not in added_indices:
#            added_indices.extend(range(row['off0'], row['off1']))
#            df_test_new.at[index, 'add'] = True

In [66]:
#df_test_new = df_test_new[df_test_new['add']]
#df_test_new = df_test_new.sort_values(by=['filename', 'off0', 'off1'])
#df_test_new = df_test_new[['filename','mark','label','off0','off1',
#                                           'span', 'code', 'semantic_rel']]
df_test_new.head()

,filename,mark,label,off0,off1,span,code,semantic_rel
0,bg_text_0.txt,T1,ENFERMEDAD,0,23,дългогодишна хипертония,UMLS:C0443252,EXACT
1,bg_text_0.txt,T1,ENFERMEDAD,26,46,има често главоболие,UMLS:C0948396,EXACT
2,bg_text_0.txt,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT
3,bg_text_0.txt,T1,ENFERMEDAD,137,147,сърцебиене,UMLS:C0030252,EXACT
4,bg_text_0.txt,T1,ENFERMEDAD,62,78,лесно заморяване,UMLS:C1859691,EXACT


In [55]:
#for index, row in df_test_new.iterrows():
#    if row['span'].endswith('.') or row['span'].endswith(','):
#        df_test_new.at[index, 'span'] = row['span'][0:-1]
#        df_test_new.at[index, 'off1'] = row['off1']-1

In [58]:
df_test_new.shape

(280, 8)

In [67]:
df_test_new.to_csv('../../distemist_eval/toy-data/subtask-linking/bg_test_entities_mistral_prompt2_elastic_search_umls.tsv', sep='\t', index=False)

In [65]:
df_gold = pd.read_csv('bg_gold_standard_entities-translated.tsv', sep='\t', header=0)
#df_gold = pd.read_csv('bg_gold_standard_entities.tsv', sep='\t', header=0)
df_gold.head()

,filename,span,off0,off1,translation,code
0,0,дългогодишна хипертония,0,23,long -standing hypertension,NaN
1,0,често главоболие,30,46,frequent headache,NaN
2,0,сърцебиене,49,59,palpitations,NaN
3,0,лесно заморяване,62,78,ease,NaN
4,0,стягане в сърдечната област,107,134,tightening in the heart area,NaN


In [66]:
df_gold.shape

(305, 6)

In [68]:
df_gold_manaul.shape

(305, 8)

In [69]:
for index, row in df_gold_manaul.iterrows():    
    df_gold.at[index, 'off0'] = row['off0']
    df_gold.at[index, 'off1'] = row['off1']
    
df_gold.head()

,filename,span,off0,off1,translation,code
0,0,дългогодишна хипертония,0,23,long -standing hypertension,NaN
1,0,често главоболие,30,46,frequent headache,NaN
2,0,сърцебиене,49,59,palpitations,NaN
3,0,лесно заморяване,62,78,ease,NaN
4,0,стягане в сърдечната област,107,134,tightening in the heart area,NaN


In [28]:
links = []
texts = []
for index, row in df_gold.iterrows():
    results = query_and_search_text(row['translation'], 'umls')
    link = ''
    text = ''
    if len(results) > 0:        
        link = 'UMLS:' + results[0]['CUI']
        text = results[0]['STR']
    
    links.append(link)
    texts.append(text)

/tmp/ipykernel_385/124867022.py:7: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  return Elasticsearch(
/tmp/ipykernel_385/124867022.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_result = elastic.search(index=index_name, body=q_dic, size=n_search, from_=0)


In [29]:
df_gold['code'] = links
df_gold['code_text'] = texts
df_gold.head()

,filename,span,off0,off1,translation,code,code_text
0,0,дългогодишна хипертония,0,23,long -standing hypertension,UMLS:C0444344,Stride long-standing
1,0,често главоболие,30,46,frequent headache,UMLS:C0948396,Frequent headache
2,0,сърцебиене,49,59,palpitations,UMLS:C0030252,Palpitations
3,0,лесно заморяване,62,78,ease,UMLS:C1331418,Ease
4,0,стягане в сърдечната област,107,134,tightening in the heart area,UMLS:C0423604,Tightening pain


In [30]:
df_gold.to_csv('bg_gold_standard_entities-translated_codes.tsv', sep='\t', index=False)

In [197]:
df_gold = pd.read_csv('bg_gold_standard_entities-manual_translated.tsv', sep='\t', header=0)
#df_gold = pd.read_csv('bg_gold_standard_entities-manual.tsv', sep='\t', header=0)

df_gold.head(30)

,filename,mark,label,off0,off1,span,true_code,semantic_rel,translation
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,UMLS:C0020538,EXACT,long -standing hypertension
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,UMLS:C0948396,EXACT,frequent headache
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT,palpitations
3,0_0,T1,ENFERMEDAD,62,78,лесно заморяване,UMLS:C1331418,EXACT,ease
4,0_0,T1,ENFERMEDAD,107,134,стягане в сърдечната област,UMLS:C0232292,EXACT,tightening in the heart area
5,0_0,T1,ENFERMEDAD,137,147,сърцебиене,UMLS:C0030252,EXACT,palpitations
6,0_0,T1,ENFERMEDAD,150,156,гадене,UMLS:C0027497,EXACT,nausea
7,0_0,T1,ENFERMEDAD,167,189,карцином на простатата,UMLS:C0496923,EXACT,prostate
8,0_0,T1,ENFERMEDAD,224,236,химиотерапия,UMLS:C0392920,EXACT,chemotherapy
9,11_0,T1,ENFERMEDAD,0,14,захарен диабет,UMLS:C0011847,EXACT,diabetes


In [198]:
df_gold = df_gold.sort_values(by=['filename', 'off0', 'off1'])

In [199]:
df_gold.shape

(305, 9)

In [200]:
df_gold_manaul = df_gold_manaul.sort_values(by=['filename', 'off0', 'off1'])

In [201]:
df_gold_manaul.shape

(305, 8)

In [202]:
df_gold_manaul.head(30)

,filename,mark,label,off0,off1,span,code,semantic_rel
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,,EXACT
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,,EXACT
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,,EXACT
3,0_0,T1,ENFERMEDAD,62,78,лесно заморяване,,EXACT
4,0_0,T1,ENFERMEDAD,107,134,стягане в сърдечната област,,EXACT
5,0_0,T1,ENFERMEDAD,137,147,сърцебиене,,EXACT
6,0_0,T1,ENFERMEDAD,150,156,гадене,,EXACT
7,0_0,T1,ENFERMEDAD,167,189,карцином на простатата,,EXACT
8,0_0,T1,ENFERMEDAD,224,236,химиотерапия,,EXACT
115,10_0,T1,ENFERMEDAD,14,24,главоболие,,EXACT


In [203]:
df_gold.head(30)

,filename,mark,label,off0,off1,span,true_code,semantic_rel,translation
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,UMLS:C0020538,EXACT,long -standing hypertension
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,UMLS:C0948396,EXACT,frequent headache
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT,palpitations
3,0_0,T1,ENFERMEDAD,62,78,лесно заморяване,UMLS:C1331418,EXACT,ease
4,0_0,T1,ENFERMEDAD,107,134,стягане в сърдечната област,UMLS:C0232292,EXACT,tightening in the heart area
5,0_0,T1,ENFERMEDAD,137,147,сърцебиене,UMLS:C0030252,EXACT,palpitations
6,0_0,T1,ENFERMEDAD,150,156,гадене,UMLS:C0027497,EXACT,nausea
7,0_0,T1,ENFERMEDAD,167,189,карцином на простатата,UMLS:C0496923,EXACT,prostate
8,0_0,T1,ENFERMEDAD,224,236,химиотерапия,UMLS:C0392920,EXACT,chemotherapy
84,10_0,T1,ENFERMEDAD,14,24,главоболие,UMLS:C0018681,EXACT,headache


In [185]:
#for index, row in df_gold.iterrows():    
#    filter_file = df_gold_manaul['filename'] == row['filename']
#    filter_span = df_gold_manaul['span'] == row['span']
#    true_rows = df_gold_manaul[filter_file & filter_span]
    
#    if true_rows.shape[0] == 0:
#        print(row)
#    true_row = true_rows.iloc[0]
#    df_gold.at[index, 'off0'] = true_row['off0']
#    df_gold.at[index, 'off1'] = true_row['off1']
    
#df_gold.head(30)

In [186]:
#df_gold['filename'] = df_gold['filename'].astype(str)

In [187]:
#for index, row in df_gold.iterrows():
#    new_val = row['filename'] + '_0'
#    df_gold.at[index, 'filename'] = new_val

In [207]:
!pip install Levenshtein


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [209]:
import Levenshtein

def select_best_candidate(text, results):
    max_ratio = 0
    max_candidate = results[0]['CUI']
    max_candidate_text = results[0]['STR']
    
    for result in results:
        ratio = Levenshtein.ratio(text, result['STR'])
        
        if ratio > max_ratio:
            max_ratio = ratio
            max_candidate = result['CUI']
            max_candidate_text = result['STR']
            
    return max_candidate, max_candidate_text

In [ ]:
links = []
texts = []
for index, row in df_gold.iterrows():
    results = query_and_search_text(row['translation'], 'umls')
    link = ''
    text = ''
    if len(results) > 0:        
        #link = 'UMLS:' + results[0]['CUI']
        link, text = select_best_candidate(row['translation'], results)
        link = 'UMLS:' + link
        #text = results[0]['STR']
    
    links.append(link)
    texts.append(text)

In [219]:
df_gold['code'] = links
df_gold['code_text'] = texts

In [220]:
df_gold['mark'] = 'T1'
df_gold['label'] = 'ENFERMEDAD'
df_gold['semantic_rel'] = 'EXACT'


In [221]:
df_gold = df_gold[['filename', 'mark', 'label', 'off0', 'off1', 'span', 'code', 'semantic_rel']]

In [222]:
df_gold = df_gold.sort_values(by=['filename', 'off0', 'off1'])

In [223]:
df_gold.head(30)

,filename,mark,label,off0,off1,span,code,semantic_rel
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,,EXACT
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,,EXACT
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,,EXACT
3,0_0,T1,ENFERMEDAD,62,78,лесно заморяване,,EXACT
4,0_0,T1,ENFERMEDAD,107,134,стягане в сърдечната област,,EXACT
5,0_0,T1,ENFERMEDAD,137,147,сърцебиене,,EXACT
6,0_0,T1,ENFERMEDAD,150,156,гадене,,EXACT
7,0_0,T1,ENFERMEDAD,167,189,карцином на простатата,,EXACT
8,0_0,T1,ENFERMEDAD,224,236,химиотерапия,,EXACT
84,10_0,T1,ENFERMEDAD,14,24,главоболие,,EXACT


In [216]:
#df_gold.to_csv('bg_gold_standard_entities-manual_norm.tsv', sep='\t', index=False)
df_gold.to_csv('bg_gold_standard_entities-predicted_lev_norm.tsv', sep='\t', index=False)

In [100]:
#df_gold_manaul = pd.read_csv('bg_text_manual.csv', header=0)

df_gold_manaul = pd.read_csv('bg_gold_standard_entities.tsv', sep='\t', header=0)
df_gold_manaul.head()

,filename,span,off0,off1
0,0,дългогодишна хипертония,0,23
1,0,често главоболие,30,46
2,0,сърцебиене,49,59
3,0,лесно заморяване,62,78
4,0,стягане в сърдечната област,107,134


In [54]:
#df_gold_manaul['span'] = list(df_gold['span'].values)

In [37]:
#df_gold_manaul = df_gold_manaul[['filename', 'span', 'off0', 'off1', 'true_code']]

In [38]:
#df_gold_manaul = df_gold_manaul.rename(columns={"true_code": "code"})

In [101]:
df_gold_manaul.head()
#filename	mark	label	off0	off1	span	code	semantic_rel

,filename,span,off0,off1
0,0,дългогодишна хипертония,0,23
1,0,често главоболие,30,46
2,0,сърцебиене,49,59
3,0,лесно заморяване,62,78
4,0,стягане в сърдечната област,107,134


In [102]:
df_gold_manaul['mark'] = 'T1'
df_gold_manaul['label'] = 'ENFERMEDAD'
df_gold_manaul['semantic_rel'] = 'EXACT'


In [103]:
df_gold_manaul['code'] = ''

In [104]:
df_gold_manaul = df_gold_manaul[['filename', 'mark', 'label', 'off0', 'off1', 'span', 'code', 'semantic_rel']]

In [105]:
df_gold_manaul['filename'] = df_gold_manaul['filename'].astype(str)

In [106]:
for index, row in df_gold_manaul.iterrows():
    #new_val = row['filename'].replace('0', '') + '0'
    #if new_val.startswith('_'):
    #    new_val = '0' + new_val
    new_val = row['filename'] + '_0'
    df_gold_manaul.at[index, 'filename'] = new_val

In [107]:
df_gold_manaul = df_gold_manaul.sort_values(by=['filename', 'off0', 'off1'])

In [108]:
df_gold_manaul.head(30)

,filename,mark,label,off0,off1,span,code,semantic_rel
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,,EXACT
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,,EXACT
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,,EXACT
3,0_0,T1,ENFERMEDAD,62,78,лесно заморяване,,EXACT
4,0_0,T1,ENFERMEDAD,107,134,стягане в сърдечната област,,EXACT
5,0_0,T1,ENFERMEDAD,137,147,сърцебиене,,EXACT
6,0_0,T1,ENFERMEDAD,150,156,гадене,,EXACT
7,0_0,T1,ENFERMEDAD,167,189,карцином на простатата,,EXACT
8,0_0,T1,ENFERMEDAD,224,236,химиотерапия,,EXACT
115,10_0,T1,ENFERMEDAD,14,24,главоболие,,EXACT


In [109]:
df_gold_manaul.to_csv('bg_gold_standard_entities-manual_ner.tsv', sep='\t', index=False)

In [16]:
df_gold_manual_norm = pd.read_csv('bg_gold_standard_entities-manual_norm.tsv', sep='\t', header=0)
df_gold_manual_norm.head()

,filename,mark,label,off0,off1,span,code,semantic_rel
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,UMLS:C0020538,EXACT
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,UMLS:C0948396,EXACT
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT
3,0_0,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT
4,0_0,T1,ENFERMEDAD,62,78,лесно заморяване,UMLS:C1331418,EXACT


In [17]:
df_gold_pred_norm = pd.read_csv('vicuna_links/distemist_evaluation_library/toy-data/subtask-linking/bg_gold_standard_entities-predicted_lev_norm.tsv', sep='\t', header=0)
df_gold_pred_norm.head()

,filename,mark,label,off0,off1,span,code,semantic_rel
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,UMLS:C0597290,EXACT
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,UMLS:C0948396,EXACT
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT
3,0_0,T1,ENFERMEDAD,62,78,лесно заморяване,UMLS:C1331418,EXACT
4,0_0,T1,ENFERMEDAD,107,134,стягане в сърдечната област,UMLS:C0011974,EXACT


In [18]:
df_gold_pred_norm['off0'] = df_gold_manual_norm['off0']
df_gold_pred_norm['off1'] = df_gold_manual_norm['off1']

In [19]:
df_gold_pred_norm.head()

,filename,mark,label,off0,off1,span,code,semantic_rel
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,UMLS:C0597290,EXACT
1,0_0,T1,ENFERMEDAD,30,46,често главоболие,UMLS:C0948396,EXACT
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT
3,0_0,T1,ENFERMEDAD,49,59,лесно заморяване,UMLS:C1331418,EXACT
4,0_0,T1,ENFERMEDAD,62,78,стягане в сърдечната област,UMLS:C0011974,EXACT


In [20]:
df_gold_pred_norm.to_csv('vicuna_links/distemist_evaluation_library/toy-data/subtask-linking/bg_gold_standard_entities-predicted_lev_norm.tsv', sep='\t', index=False)

In [27]:
df_pred_sap = pd.read_csv('vicuna_links/distemist_evaluation_library/toy-data/subtask-linking/bg_test_entities_sap_link.tsv', sep='\t', header=0)

In [29]:
df_pred_sap.head(50)

,filename,mark,label,off0,off1,span,code,semantic_rel
0,0_0,T1,ENFERMEDAD,0,23,дългогодишна хипертония,UMLS:C0745114,EXACT
1,0_0,T1,ENFERMEDAD,36,46,главоболие,UMLS:C0018681,EXACT
2,0_0,T1,ENFERMEDAD,49,59,сърцебиене,UMLS:C0030252,EXACT
3,0_0,T1,ENFERMEDAD,68,78,заморяване,UMLS:C0232292,EXACT
4,0_0,T1,ENFERMEDAD,107,134,стягане в сърдечната област,UMLS:C0027497,EXACT
5,0_0,T1,ENFERMEDAD,137,147,сърцебиене,UMLS:C0042571,EXACT
6,0_0,T1,ENFERMEDAD,150,156,гадене,UMLS:C0376358,EXACT
7,0_0,T1,ENFERMEDAD,167,189,карцином на простатата,UMLS:C0013217,EXACT
8,10_0,T1,ENFERMEDAD,14,24,главоболие,UMLS:C0018681,EXACT
9,10_0,T1,ENFERMEDAD,27,40,виене на свят,UMLS:C0042571,EXACT


In [41]:
df_pred_sap_nonum = df_pred_sap[~df_pred_sap['span'].str.contains("1|10|19|20|3|4|5|6|7|8|9|0")]

In [42]:
df_pred_sap_nonum.to_csv('vicuna_links/distemist_evaluation_library/toy-data/subtask-linking/bg_test_entities_sap_link_nonum.tsv', sep='\t', index=False)